# Creating a UGrid

Below are the imports that we will be using to work with ugrids

In [1]:
from xms.grid.ugrid import UGrid
from xms.grid.ugrid import ugrid_utils

And here are some general imports we will be using

In [2]:
import time

<p>
Ugrids are the basic geometry that make up a grid.  They are made up of cells which are made up of points. <br>
Some important terminology concering UGrids are:<br>
    - Location: The actual x,y,z coordinates of a point/vertex in the UGrid.<br>
    - Point: The integer index of the point inside the list of locations (UGrid.locations)<br>
    - Cell: A group of locations that make up a shape.<br>
</p>

In [3]:
locations = [(5.0, 74.0, 0.0),
          (22.0, 74.0, 0.0),
          (23.63, 60.57, 0.0),
          (11.35, 55.51, 0.0),
          (2.86, 62.19, 0.0),
          (12.53, 66.17, 0.0)]

The code above creates a list of locations. Each location in the list is a triple
where each value in the triple corresponds to x,y,z coordinates (a point).  This ugrid is going
to be 3D, but we could make a 2D ugrid if we wanted to.

In [4]:
cellstream = [
    UGrid.cell_type_enum.TRIANGLE, 3, 0, 4, 5,
    UGrid.cell_type_enum.TRIANGLE, 3, 5, 4, 3,
    UGrid.cell_type_enum.TRIANGLE, 3, 5, 2, 1,
    UGrid.cell_type_enum.TRIANGLE, 3, 2, 5, 3,
    UGrid.cell_type_enum.TRIANGLE, 3, 5, 1, 0
]

The code above creates a list of cells.  Each five values in the list together represent a cell.

The first cell in the list is `UGrid.cell_type_enum.TRIANGLE, 3, 0, 4, 5`.

The first value that describes the cell is `UGrid.cell_type_enum.TRIANGLE`, which is the type of the cell.  The cell type determines how the points in a cell relate to each other and what shape the cell will take.

The next value, `3`, indicates how many points there are in the cell.  Because we put `3`, there should be three values that follow this value.

The next three values `0, 4, 5` are each indicies that tell the indexes in `locations` that hold the points that make up the cell.  The following code illustrates this.


In [5]:
index_point_1 = cellstream[2]
index_point_2 = cellstream[3]
index_point_3 = cellstream[4]
print(locations[index_point_1], locations[index_point_2], locations[index_point_3])

(5.0, 74.0, 0.0) (2.86, 62.19, 0.0) (12.53, 66.17, 0.0)


Now that we have created the locations and cells that will make up our UGrid, we can make a Ugrid by passing
the locations and cellstream lists to the UGrid constructor:

In [6]:
out_ugrid = UGrid(locations, cellstream)

There are many operations that can be performed on a UGrid.  Lets write out our UGrid to a file so we can import it into GMS and see what it looks like.

# Writing a UGrid out to a File

In [7]:
out_file_name = "output_files/test.xmugrid"
ugrid_utils.write_ugrid_to_ascii_file(out_ugrid, out_file_name)

When we take the .xmugrid file that we just created and import it into GMS we get this:

![The ugrid generated by our code imported into GMS](images/ugrid.png)

# Reading a UGrid in from a File

Just as we wrote out the UGrid to a file, we can read in a UGrid from a file.

In [8]:
in_ugrid = ugrid_utils.read_ugrid_from_ascii_file(out_file_name)
print(f'out_ugrid: {out_ugrid}')
print(f'in_ugrid: {in_ugrid}')

out_ugrid: <UGrid - Number of Locations: 6, Number of Cells: 5, Extents: ((2.86, 55.51, 0.0), (23.63, 74.0, 0.0)), Modified: True>
in_ugrid: <UGrid - Number of Locations: 6, Number of Cells: 5, Extents: ((2.86, 55.51, 0.0), (23.63, 74.0, 0.0)), Modified: True>


# Important Note on UGrid.locations
The UGrid `in_ugrid` that we read in has the same data as the UGrid `out_ugrid` that we created.
We can get at that data, but there is an important note on getting UGrid.locations.  Each time that
you call the UGrid.locations property, a new array is created and returned with a copy of the locations data.

In [9]:
in_ugrid.locations

array([[ 5.  , 74.  ,  0.  ],
       [22.  , 74.  ,  0.  ],
       [23.63, 60.57,  0.  ],
       [11.35, 55.51,  0.  ],
       [ 2.86, 62.19,  0.  ],
       [12.53, 66.17,  0.  ]])

When there are a lot of cells in the UGrid this can be expensive.  If we call to it multiple times watch how much time it takes.

In [10]:
def get_time():
    return time.perf_counter_ns()

_iterations = 100000

In [11]:
# Calling UGrid.locations each time
time_start = get_time()

for i in range(_iterations):
    _locations = in_ugrid.locations
    for cell in _locations:
        cell

time_end = get_time()

time_calling_each_time = time_end - time_start

In [12]:
# Calling UGrid.locations once
time_start = get_time()

_locations = in_ugrid.locations

for i in range(_iterations):
    for cell in _locations:
        cell

time_end = get_time()

time_calling_once = time_end - time_start

In [13]:
print(f'Calling UGrid.locations each time: {time_calling_each_time}')
print(f'Calling UGrid.locations once: {time_calling_once}')
print(f'Difference (calling_each_time - calling_once): {time_calling_each_time - time_calling_once}')
print(f'Calling UGrid.locations each time took {time_calling_each_time / time_calling_once} times longer in this scenario')

Calling UGrid.locations each time: 243230700
Calling UGrid.locations once: 117866100
Difference (calling_each_time - calling_once): 125364600
Calling UGrid.locations each time took 2.063618801334735 times longer in this scenario


It is important to not make more calls to UGrid.locations than necessary as this can be a slow process.

# UGrid Operations

Lets take a look at some of the operations available to UGrids.  First we'll look at operations related to locations/points.

#### UGrid.point_count
We can get the number of points with the property UGrid.point_count:

In [14]:
in_ugrid.point_count

6

#### UGrid.get_point_location(index)
We can get the actual x,y,z coordinates of a point (the point location) with the `UGrid.get_point_location(index)` method. It takes an index and returns a Tuple.

In [15]:
in_ugrid.get_point_location(0)

(5.0, 74.0, 0.0)

The index is 0 based and goes up to `UGrid.point_count - 1`. If you pass it an index outside that range it returns a Tuple filled with zeros:

In [16]:
print(in_ugrid.get_point_location(-1))
in_ugrid.get_point_location(6)

(0.0, 0.0, 0.0)


(0.0, 0.0, 0.0)

#### UGrid.set_point_location(index, location)
We can set the location of a point with the `UGrid.set_point_location(index, location)` method. It takes an index, a Tuple, and returns a Boolean indicating whether the method was successful or not.

In [17]:
result = in_ugrid.set_point_location(0,(8.0,74.0,0.0))
result

True

In [18]:
result = in_ugrid.set_point_location(-1,(0,0,0))
result

False

#### Other Methods
Here are two methods that are similar to the previous two methods:

In [19]:
_r1 = in_ugrid.get_point_xy0(0)              # The same as get_point_location(), except the z coordinate is set to 0 in the returned Tuple
_r2 = in_ugrid.get_points_locations((0,1))   # The same as get_point_location(), except it can be passed multiple indices and returns a list of results

print(f'get_point_xy0(0): {_r1}')
print(f'get_points_locations((0,1)): {_r2}')

get_point_xy0(0): (8.0, 74.0, 0.0)
get_points_locations((0,1)): [[ 8. 74.  0.]
 [22. 74.  0.]]


## Cell Methods and Properties
Here are some methods and properties releated to cells:<br>

#### UGrid.cell_count
We can get the number of cells in the UGrid with UGrid.cell_count:

In [20]:
in_ugrid.cell_count

5

#### UGrid.get_cell_points(index)
Gets the indices for the locations in UGrid.locations that make up the cell at the given index

In [21]:
in_ugrid.get_cell_points(0)   # Gets the points of a cell (including polyhedron)

(0, 4, 5)

#### UGrid.get_cell_locations(index)
Gets the actual x,y,z coordinates of the points that make up the cell at the given index

In [22]:
in_ugrid.get_cell_locations(0)

array([[ 8.  , 74.  ,  0.  ],
       [ 2.86, 62.19,  0.  ],
       [12.53, 66.17,  0.  ]])

#### UGrid.get_cell_dimension(index) 
Gets the number of dimensions of the cell at the given index

In [23]:
in_ugrid.get_cell_dimension(0)

2

## Cell Edges
You can simply iterate over the edges of a cell with the following code:

In [24]:
edges = in_ugrid.get_cell_edges(0)

for edge in edges:
    print(edge)

(0, 4)
(4, 5)
(5, 0)


Each edge tuple holds the indices of the locations in UGrid.locations that make up the edge. <br>
There are useful methods that can tell you information about neighboring cells, such as `UGrid.get_edge_adjacent_cells(edge)`

In [25]:
_r1 = in_ugrid.get_edge_adjacent_cells((0, 4))
_r2 = in_ugrid.get_edge_adjacent_cells((4, 5))
print(_r1)
print(_r2)

(0,)
(0, 1)


This will return a tuple of indexes of the cells that are adjacent to the given cells. <br>
There are a lot more UGrid methods that we don't have the time to cover right now, just know there are many useful methods.